## Deep Classification Network

0. First, the grid established in the Dataset exploration notebook will be tested with the SVM. Then, a DNN is implemented.

1. Import all `modules` we are going to need

In [1]:
import torch
from torch import nn
import torchvision

2. Set up y. Then, import and process X.

In [3]:
import os
import pandas as pd

In [4]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/mat2csv/emotic_pre/')
train = pd.read_csv('train.csv')
train_mscoco = train[train['Folder'] == 'mscoco/images']
train_frames = train[train['Folder'] == 'framesdb/images']

In [5]:
import re

In [22]:
y = []
y_fr = []
for rating_ms in train_mscoco['Continuous_Labels']:
    y.append(rating_ms)

for rating_frames in train_frames['Continuous_Labels']: 
    y.append(rating_frames)

In [23]:
len(y)

21624

In [24]:
y_ = []
for idx, label in enumerate(y):
    s = (re.sub(r"[\[\]]",'',y[idx]))
    for i, x in enumerate(s):
        y_.append(x)
        if x == ',':
            y_.remove(x)
        if x == ' ':
            y_.remove(x)
        try:
            if x == '1' and s[i+1] == '0':
                y_.remove(x)
                y_.append('10')
        except IndexError:
            continue
        



In [25]:
for idx, el in enumerate(y_):
    if el == '10':
        if y_[idx+1] == '0':
            y_.remove(y_[idx+1])



In [26]:
len(y_)

64872

In [28]:
cont_labels = []
for val in y_:
    cont_labels.append(float(val))

In [29]:
img = 0
Arousal = []
Valence = []
Dominance = []
for val in cont_labels:
    if img == 0:
        Arousal.append(val)
        img = img+1
        continue
    if img == 1:
        Valence.append(val)
        img = img+1
        continue 
    if img == 2:
        Dominance.append(val)
        img = 0
        continue
    



In [30]:
import numpy as np

Arousal = np.array(Arousal)



In [31]:
Arousal.shape

(21624,)

In [32]:
y = Arousal

In [33]:
from sklearn.feature_selection import SelectKBest

In [34]:
k_select = SelectKBest(k=30000)

In [42]:
y_df = pd.DataFrame(y)

In [43]:
y_df.to_csv('Arousal_frames_emodb.csv')

In [44]:
y_df

,0
0,5.0
1,6.0
2,7.0
3,8.0
4,7.0
...,...
21619,6.0
21620,5.0
21621,5.0
21622,6.0


In [18]:
y_series = y_df.squeeze()

In [19]:
y_series[12000:14276]

12000    7.0
12001    6.0
12002    5.0
12003    6.0
12004    8.0
        ... 
14271    5.0
14272    8.0
14273    2.0
14274    6.0
14275    7.0
Name: 0, Length: 2276, dtype: float64

**NEXT STEP**:

`Rename`all columns more meaningful. Then apply k-best to all chunked dataframes.

In [37]:
os.chdir("/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis")

In [38]:
# the number of row in each data frame
# you can put any value here according to your situation
chunksize = 5000
# the list that contains all the dataframes
list_of_dataframes = []

print('processing...')
for index, df in enumerate(pd.read_csv('X_Matrix.csv', chunksize=chunksize)):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)
    print('Read the {num} dataframe'.format(num=index))
    
   

processing...
Read the 0 dataframe
Read the 1 dataframe
Read the 2 dataframe
Read the 3 dataframe
Read the 4 dataframe


In [39]:
# the number of row in each data frame
# you can put any value here according to your situation
chunksize = 2000

print('processing...')
for index, df in enumerate(pd.read_csv('X_frames.csv', chunksize=chunksize)):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)
    print('Read the {num} dataframe'.format(num=index))
    

processing...
Read the 0 dataframe
Read the 1 dataframe
Read the 2 dataframe
Read the 3 dataframe


In [41]:
len(list_of_dataframes)

9

**TESTING IF REGULAR SGD WORKS ON ALL FEATURES WITH 21 K SAMPLES**

In [45]:
from sklearn.linear_model import SGDClassifier

In [46]:
from sklearn.preprocessing import StandardScaler

In [47]:
scaler = StandardScaler()

In [49]:
Log = SGDClassifier(loss='log_loss')

In [64]:
list_of_dataframes[0].shape[0]

3000

In [ ]:
i=0
j=3000
c_initial=0
for chunk, df in enumerate(list_of_dataframes):
    X = scaler.fit_transform(df)
    print('Partial fitting the model...')
    Log.partial_fit(X,y[c_initial:df.shape[0]], classes = np.unique(y))
    if chunk == 7:
        c_initial = list_of_dataframes[i].shape[0]
        X_pred_prob = Log.score(X,y[c_initial:df.shape[0]])
        print('The accuracy for the training set is {prob}'.format(prob=X_pred_prob))
        break
    else:
        del X
        c_initial = list_of_dataframes[i].shape[0]
        i=i+1
        j=j+df.shape[0]
        print('Will now continue with chunk {chunk}'.format(chunk=chunk))

**TESTING END**

In [22]:
for df in list_of_dataframes:
    for pos, col in enumerate(df):
        df.columns.values[pos] = 'Feature ' + str(pos)
        

In [23]:
k_list = []
track = 0
i=0
j=3000
for chunk in list_of_dataframes:
    print('processing...')
    k_select.fit_transform(chunk, y_df[i:j])
    cols_idxs = k_select.get_support(indices=True)
    k_list.append(chunk.iloc[:,cols_idxs])
    print('Applied k-best to {track}/5 chunks!'.format(track=track))
    track=track+1
    if track == 4:
        i=12000
        j=12000+list_of_dataframes[4].shape[0]
        
    else:
        i=i+3000
        j=j+3000
        

processing...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 0/5 chunks!
processing...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 1/5 chunks!
processing...


...

Applied k-best to 2/5 chunks!
processing...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 3/5 chunks!
processing...


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 4/5 chunks!


**Step 4.**

`Majority voting`

In [24]:
len(set(k_list[0]).intersection(k_list[1], k_list[2]))

25390

In [25]:
features = list(set(k_list[0]).intersection(k_list[1], k_list[2]))

**Step 5.**

Reduce columns from *k* (=40000) to the amount of features that are commmon between dataframe one, two and three.

Then concatenate.

In [26]:
feature_list = []
for chunk, df in enumerate(list_of_dataframes):
    print('processing chunk {chunk}/2'.format(chunk=chunk))
    feature_list.append(df[features])
    if chunk == 2:
        break

processing chunk 0/2
processing chunk 1/2
processing chunk 2/2


In [28]:
for chunk, df in enumerate(feature_list):
    print('df_{cunk}'.format(cunk=chunk))

df_0
df_1
df_2


In [29]:
feature_list[0].to_csv('X_1.csv')
feature_list[1].to_csv('X_2.csv')
feature_list[2].to_csv('X_3.csv')

In [32]:
# remaining dataframes for RAM purposes
list_df_r = []
list_df_r.append(list_of_dataframes[3])
list_df_r.append(list_of_dataframes[4])

In [37]:
for pos, df in enumerate(list_df_r):
    print('processing chunk {pos}/1'.format(pos=pos))
    feature_list.append(df[features])
    

processing chunk 0/1
processing chunk 1/1


In [41]:
feature_list[3].to_csv('X_4.csv')
feature_list[4].to_csv('X_5.csv')